In [11]:
import json
from datetime import datetime, timedelta
import requests
from google.colab import userdata

my_secret_value = userdata.get('GOOGLE_API_KEY')

GEMINI_API_KEY = my_secret_value
GEMINI_API_URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"

# This mock data simulates Nifty 50 OHLCV for the last two days.
def get_mock_nifty_data():
    """
    Generates mock Nifty 50 data for the last two trading days.
    """
    today = datetime.now()
    # Simulate last two trading days (excluding weekends for simplicity)
    day1 = today - timedelta(days=1)
    day2 = today - timedelta(days=2)

    # Adjust for weekends
    if day1.weekday() >= 5: # Saturday or Sunday
        day1 -= timedelta(days=day1.weekday() - 4) # Move to Friday
    if day2.weekday() >= 5: # Saturday or Sunday
        day2 -= timedelta(days=day2.weekday() - 4) # Move to Friday

    # Example data
    data = [
        {
            "date": day2.strftime("%Y-%m-%d"),
            "open": 22000.00,
            "high": 22150.00,
            "low": 21950.00,
            "close": 22100.00,
            "volume": 250000000
        },
        {
            "date": day1.strftime("%Y-%m-%d"),
            "open": 22100.00,
            "high": 22250.00,
            "low": 22050.00,
            "close": 22200.00,
            "volume": 270000000
        }
    ]
    print(f"--- Mock Nifty 50 Data Loaded ---")
    for d in data:
        print(f"Date: {d['date']}, Open: {d['open']}, Close: {d['close']}")
    print(f"---------------------------------")
    return data

# --- Data Fetching
def fetch_nifty_data(days_ago=2):
    return get_mock_nifty_data()

# --- Prompt Engineering
def prepare_llm_prompt(nifty_data):
    if not nifty_data or len(nifty_data) < 2:
        return "Insufficient data to make a prediction."

    # Sort data by date
    nifty_data_sorted = sorted(nifty_data, key=lambda x: x['date'])

    prompt_parts = ["Analyze the following Nifty 50 market data for the last two trading days:"]

    for i, day_data in enumerate(nifty_data_sorted):
        prompt_parts.append(
            f"\nDay {i+1} ({day_data['date']}): "
            f"Open: {day_data['open']:.2f}, "
            f"High: {day_data['high']:.2f}, "
            f"Low: {day_data['low']:.2f}, "
            f"Close: {day_data['close']:.2f}, "
            f"Volume: {day_data['volume']}"
        )

    prompt_parts.append(
        "\n\nBased on this data, what is your qualitative prediction for today's Nifty 50 market trend? "
        "Please categorize it as 'Bullish', 'Bearish', or 'Sideways'. "
        "Provide a brief explanation for your prediction. "
        "Format your response as: 'Prediction: [Trend]\nReason: [Explanation]'."
    )

    return "".join(prompt_parts)

# --- LLM Interaction
async def get_llm_prediction(prompt):
    chat_history = []
    chat_history.append({"role": "user", "parts": [{"text": prompt}]})

    payload = {"contents": chat_history}

    try:
        headers = {'Content-Type': 'application/json'}
        api_url_with_key = f"{GEMINI_API_URL}?key={GEMINI_API_KEY}"
        response = requests.post(api_url_with_key, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        result = response.json()

        if result.get("candidates") and len(result["candidates"]) > 0 and \
           result["candidates"][0].get("content") and \
           result["candidates"][0]["content"].get("parts") and \
           len(result["candidates"][0]["content"]["parts"]) > 0:
            text = result["candidates"][0]["content"]["parts"][0]["text"]
            return text
        else:
            return f"Error: Unexpected LLM response structure: {json.dumps(result, indent=2)}"

    except requests.exceptions.HTTPError as http_err:
        return f"HTTP error occurred: {http_err} - Response: {response.text}"
    except requests.exceptions.ConnectionError as conn_err:
        return f"Connection error occurred: {conn_err}. Check your internet connection or API endpoint."
    except requests.exceptions.Timeout as timeout_err:
        return f"Timeout error occurred: {timeout_err}. The request took too long."
    except requests.exceptions.RequestException as req_err:
        return f"An error occurred during the API request: {req_err}"
    except json.JSONDecodeError as json_err:
        return f"Failed to decode JSON response: {json_err} - Response text: {response.text}"
    except Exception as e:
        return f"An unexpected error occurred: {e}"

# --- Main Execution
async def main():
    print("Starting Nifty 50 Market Prediction...")
    nifty_data = fetch_nifty_data(days_ago=2)

    if not nifty_data:
        print("Could not retrieve Nifty 50 data. Exiting.")
        return

    prompt = prepare_llm_prompt(nifty_data)
    print("\n--- LLM Prompt ---")
    print(prompt)
    print("------------------")

    print("\nRequesting prediction from LLM... (This may take a moment)")
    prediction = await get_llm_prediction(prompt)

    print("\n--- Today's Nifty 50 Market Prediction ---")
    print(prediction)
    print("------------------------------------------")
    # print("\nDisclaimer: This prediction is for educational purposes only and not financial advice.")

await main()

Starting Nifty 50 Market Prediction...
--- Mock Nifty 50 Data Loaded ---
Date: 2025-06-27, Open: 22000.0, Close: 22100.0
Date: 2025-06-27, Open: 22100.0, Close: 22200.0
---------------------------------

--- LLM Prompt ---
Analyze the following Nifty 50 market data for the last two trading days:
Day 1 (2025-06-27): Open: 22000.00, High: 22150.00, Low: 21950.00, Close: 22100.00, Volume: 250000000
Day 2 (2025-06-27): Open: 22100.00, High: 22250.00, Low: 22050.00, Close: 22200.00, Volume: 270000000

Based on this data, what is your qualitative prediction for today's Nifty 50 market trend? Please categorize it as 'Bullish', 'Bearish', or 'Sideways'. Provide a brief explanation for your prediction. Format your response as: 'Prediction: [Trend]
Reason: [Explanation]'.
------------------

Requesting prediction from LLM... (This may take a moment)

--- Today's Nifty 50 Market Prediction ---
Prediction: Bullish
Reason: The Nifty 50 closed higher for two consecutive days, with higher highs and h